In [14]:
import numpy as np
from glob import glob
from os.path import join
import seaborn as sns
import matplotlib.pyplot as plt 
%matplotlib inline

In [4]:
in_dirs = ['../data/000.5pc','../data/001.9pc', '../data/007.1pc', '../data/026.6pc', '../data/100.0pc']
out_dirs = [x+'-split' for x in in_dirs]

files = ['quad_well_{:02d}.npy'.format(x) for x in range(100)]

In [5]:
tau = 25

In [6]:
def split_and_save(tau, in_dir, out_dir, fname, offset):
    x = np.load(join(in_dir, fname))
    for i in range(x.shape[0]-tau):
        out = np.zeros((2, 1))
        out[[0, 1], :] = x[[i,i+tau],:]
        np.save(join(out_dir, 'traj-{:06d}.npy'.format(offset*(x.shape[0]-tau)+i)), out)

In [7]:
for i, in_dir in enumerate(in_dirs[:1]):
    out_dir = out_dirs[i]
    print(out_dir)
    for j, fname in enumerate(files):
        print('\t', fname, end=', ')
        split_and_save(tau, in_dir, out_dir, fname, j)

../data/000.5pc-split
	 quad_well_00.npy, 	 quad_well_01.npy, 	 quad_well_02.npy, 	 quad_well_03.npy, 	 quad_well_04.npy, 	 quad_well_05.npy, 	 quad_well_06.npy, 	 quad_well_07.npy, 	 quad_well_08.npy, 	 quad_well_09.npy, 	 quad_well_10.npy, 	 quad_well_11.npy, 	 quad_well_12.npy, 	 quad_well_13.npy, 	 quad_well_14.npy, 	 quad_well_15.npy, 	 quad_well_16.npy, 	 quad_well_17.npy, 	 quad_well_18.npy, 	 quad_well_19.npy, 	 quad_well_20.npy, 	 quad_well_21.npy, 	 quad_well_22.npy, 	 quad_well_23.npy, 	 quad_well_24.npy, 	 quad_well_25.npy, 	 quad_well_26.npy, 	 quad_well_27.npy, 	 quad_well_28.npy, 	 quad_well_29.npy, 	 quad_well_30.npy, 	 quad_well_31.npy, 	 quad_well_32.npy, 	 quad_well_33.npy, 	 quad_well_34.npy, 	 quad_well_35.npy, 	 quad_well_36.npy, 	 quad_well_37.npy, 	 quad_well_38.npy, 	 quad_well_39.npy, 	 quad_well_40.npy, 	 quad_well_41.npy, 	 quad_well_42.npy, 	 quad_well_43.npy, 	 quad_well_44.npy, 	 quad_well_45.npy, 	 quad_well_46.npy, 	 quad_well_47.npy, 	 quad_well_48.npy

In [8]:
from pyemma.msm import MaximumLikelihoodMSM
from msmbuilder.cluster import NDGrid

mm =  MaximumLikelihoodMSM(connectivity='largest', count_mode='sliding',
           dt_traj='1 step', lag=1, maxerr=1e-08, maxiter=1000000,
           mincount_connectivity='1/n', reversible=True, score_k=4,
           score_method='VAMP2', sparse=False, statdist_constraint=None)

/Users/robert_arbon/anaconda/envs/ml4dyn/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/robert_arbon/anaconda/envs/ml4dyn/lib/python3.5/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/Users/robert_arbon/anaconda/envs/ml4dyn/lib/python3.5/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pa

In [9]:
paths = glob('../data/000.5pc-split/*npy')

In [35]:
split_X = [np.load(x) for x in paths]

In [36]:
X = [np.load(x) for x  in glob('../data/000.5pc/*npy')]

In [37]:
cluster = NDGrid(min=-1.2, max=1.2, n_bins_per_feature=90)

In [38]:
split_dtrajs = cluster.fit_transform(split_X)

In [39]:
dtrajs = cluster.fit_transform(X)

In [40]:
split_mm =  MaximumLikelihoodMSM(connectivity='largest', count_mode='sliding',
           dt_traj='1 step', lag=1, maxerr=1e-08, maxiter=1000000,
           mincount_connectivity='1/n', reversible=True, score_k=4,
           score_method='VAMP2', sparse=False, statdist_constraint=None)

In [41]:
mm =  MaximumLikelihoodMSM(connectivity='largest', count_mode='sliding',
           dt_traj='1 step', lag=25, maxerr=1e-08, maxiter=1000000,
           mincount_connectivity='1/n', reversible=True, score_k=4,
           score_method='VAMP2', sparse=False, statdist_constraint=None)

In [42]:
split_mm.fit(split_dtrajs)

MaximumLikelihoodMSM(connectivity='largest', count_mode='sliding',
           dt_traj='1 step', lag=1, maxerr=1e-08, maxiter=1000000,
           mincount_connectivity='1/n', reversible=True, score_k=4,
           score_method='VAMP2', sparse=False, statdist_constraint=None)

In [43]:
mm.fit(dtrajs)

MaximumLikelihoodMSM(connectivity='largest', count_mode='sliding',
           dt_traj='1 step', lag=25, maxerr=1e-08, maxiter=1000000,
           mincount_connectivity='1/n', reversible=True, score_k=4,
           score_method='VAMP2', sparse=False, statdist_constraint=None)

In [45]:
mm.pi.shape

(81,)

In [47]:
split_mm.pi.shape

(81,)

In [48]:
mm.timescales(k=10)

array([ 977.00568357,  125.41846284,   66.08464041,   11.6731514 ,
         10.40267202,    9.26464985,    9.17419948,    8.88113523,
          8.78393438,    8.69615223])

In [50]:
split_mm.timescales(k=10)*25

array([ 977.00568357,  125.41846284,   66.08464041,   11.6731514 ,
         10.40267202,    9.26464985,    9.17419948,    8.88113523,
          8.78393438,    8.69615223])